CRITIC IMPLEMENTATION

In [1]:
import pandas as pd
import numpy as np

In [2]:
mobile_full=pd.read_excel('edas_data_python.xlsx')
mobile_full.head()

,Alternatives,P/E RATİO,M/B RATIO,EPS,DIV. YIELD,DIV. PAYOUT
0,AEFES,15.596684,0.426024,1.804230,0.063754,0.911525
1,AVOD,15.107937,1.902159,0.125100,0.064819,0.000000
2,BANVT,109.431308,8.433842,0.951282,0.009068,0.000000
3,CCOLA,14.436836,2.225982,8.928549,0.068157,0.266561
4,DARDL,12.916062,9.940285,0.274077,0.077423,0.000000


REFERENCE TABLE

In [3]:
criteria=['P/E RATIO','M/B RATIO','EPS','DIV. YIELD','DIV. PAYOUT']
types=['cost','cost','benefit','benefit','cost']
mobile_full_v2=mobile_full.iloc[:,1:6]
mobile_full_v2.head()
mobile_full_np=np.array(mobile_full_v2)
print(mobile_full_np)

[[ 1.55966845e+01  4.26023631e-01  1.80422962e+00  6.37536967e-02
   9.11524778e-01]
 [ 1.51079369e+01  1.90215900e+00  1.25099808e-01  6.48185531e-02
   0.00000000e+00]
 [ 1.09431308e+02  8.43384206e+00  9.51281692e-01  9.06846227e-03
   0.00000000e+00]
 [ 1.44368359e+01  2.22598214e+00  8.92854925e+00  6.81568645e-02
   2.66560662e-01]
 [ 1.29160621e+01  9.94028465e+00  2.74077345e-01  7.74229787e-02
   0.00000000e+00]
 [ 3.80000000e+00  1.76000000e+00  1.65000000e+00  2.63157895e-01
   0.00000000e+00]
 [ 2.69289455e+01  4.42461929e+00  6.96648148e-01  3.59653148e-02
   1.07658364e+00]
 [ 7.55373593e+01  3.50904083e+00  5.42777778e-02  1.33360633e-02
   0.00000000e+00]
 [ 1.62857737e+01  2.51100535e+00  3.86840694e-01  6.03495622e-02
   0.00000000e+00]
 [ 1.44992930e+01  1.90044044e+00  6.76584713e-01  6.87586091e-02
   1.67458705e-01]
 [ 1.88192144e+01  5.00493626e+00  3.84181818e-01  4.83247570e-02
   0.00000000e+00]
 [ 1.22835003e+02  5.78659969e+00  5.51145833e-02  7.40787410e-03

In [6]:
len(types)

5

STEP 1 ) R- & R+

In [9]:
pos_r=[]
neg_r=[]
for k in range(len(types)):
  if types[k]=='benefit':
    m=np.max(mobile_full_np,axis=0)
    n=np.min(mobile_full_np,axis=0)
  else :
    m=np.min(mobile_full_np,axis=0)
    n=np.max(mobile_full_np,axis=0)
pos_r.append(m)
neg_r.append(n)
print('r+ : ', pos_r)
print('r- : ', neg_r)

r+ :  [array([-55.4935029 ,   0.42602363,  -1.35769591,  -0.08464438,
        -0.59569444])]
r- :  [array([211.82347862,   9.94028465,   8.92854925,   0.26315789,
        11.45696456])]


In [38]:
neg_r[0][0]

211.82347861842103

STEP 2) NORMALIZATION

In [39]:
normal_table=np.zeros(shape=(mobile_full_np.shape[0],mobile_full_np.shape[1]))
for k in range(mobile_full_np.shape[1]):
  for l in range(mobile_full_np.shape[0]):
    m=(mobile_full_np[l][k]-neg_r[0][k])/(pos_r[0][k]-neg_r[0][k])
    normal_table[l][k]=+m
print(normal_table)     

In [49]:
normal_table.shape

(27, 5)

In [58]:
X = normal_table[:, [1]]
X

array([[ 1.        ],
       [ 0.84485023],
       [ 0.15833522],
       [ 0.81081468],
       [-0.        ],
       [ 0.85979191],
       [ 0.57972609],
       [ 0.67595831],
       [ 0.78085721],
       [ 0.84503086],
       [ 0.51873166],
       [ 0.43657463],
       [ 0.76479295],
       [ 0.85022918],
       [ 0.71243139],
       [ 0.71186147],
       [ 0.90164595],
       [ 0.95995779],
       [ 0.81091142],
       [ 0.97551366],
       [ 0.87265558],
       [ 0.76269063],
       [ 0.78403256],
       [ 0.82343358],
       [ 0.52822854],
       [ 0.91112271],
       [ 0.85015304]])

STEP 3) Correlation Matrix

In [68]:
#x=normal_table[:, [0]]
#y=normal_table[:, [1]]
corr_mat=pd.DataFrame(normal_table)
corr_mat_son=corr_mat.corr()
corr_mat_son

,0,1,2,3,4
0,1.000000,0.101119,-0.165792,-0.269337,0.515307
1,0.101119,1.000000,-0.146008,-0.147629,-0.199341
2,-0.165792,-0.146008,1.000000,0.434081,-0.048266
3,-0.269337,-0.147629,0.434081,1.000000,-0.111502
4,0.515307,-0.199341,-0.048266,-0.111502,1.000000


STEP 4) Standard Deviation

In [69]:
std_mat=corr_mat.std()
print(std_mat)

0    0.242233
1    0.234332
2    0.186939
3    0.210557
4    0.183289
dtype: float64


STEP 5) Index Criteria

In [87]:
index_criteria_list=[]
for i in range(len(std_mat)):
  x_list=[]
  for j in range(corr_mat_son.shape[0]):
    y=+corr_mat_son.iloc[0,j]
    x=1-y
    x_list.append(x)
  z=sum(x_list)*std_mat[i]
  index_criteria_list.append(z)


STEP 6) WEIGHTS

In [89]:
weight_list=[]
for i in range(len(index_criteria_list)):
  y=index_criteria_list[i]/sum(index_criteria_list)
  weight_list.append(y)


In [90]:
weight_list

[0.22909458417299952,
 0.2216221304340067,
 0.17679933919469204,
 0.19913659135939563,
 0.173347354838906]